In [1]:
#@title 🔧 **Repository Setup & Module Loading**
import os
import sys
import subprocess
import shutil
from pathlib import Path
import importlib

# Configuration
REPO_URL = "https://github.com/remphanostar/SD-LongNose.git"
REPO_DIR = "/content/SD-LongNose"
MODULE_PATH = "/content/SD-LongNose/github_upload"

print("🚀 PINOKIO AI APP MANAGER - IPYWIDGETS INTERFACE")
print("=" * 60)

# Fresh repo clone
if os.path.exists(REPO_DIR):
    print("🗑️ Removing existing repository...")
    shutil.rmtree(REPO_DIR)

print("📥 Cloning repository from GitHub...")
subprocess.run(["git", "clone", "--depth", "1", REPO_URL, REPO_DIR], check=True)
print("✅ Repository cloned successfully")

# Add to Python path
if MODULE_PATH not in sys.path:
    sys.path.insert(0, MODULE_PATH)

# Import the real modules
print("🔌 Importing modules from GitHub repository...")
from pinokio_cloud_main import PinokioCloudGPU
from modules.pinokio_controller import PinokioController
from modules.pinokio_installer import PinokioInstaller
from modules.platform_detector import PlatformDetector
from modules.tunnel_manager import TunnelManager

print("✅ All modules loaded from repository")
print("📁 Source: https://github.com/remphanostar/SD-LongNose")

🚀 PINOKIO AI APP MANAGER - IPYWIDGETS INTERFACE
🗑️ Removing existing repository...
📥 Cloning repository from GitHub...
Cloning into '/content/SD-LongNose'...

✅ Repository cloned successfully
🔌 Importing modules from GitHub repository...
✅ All modules loaded from repository
📁 Source: https://github.com/remphanostar/SD-LongNose


In [2]:
#@title 📦 **Dependencies & System Setup**
import subprocess
import sys

# Install Python packages
packages = [
    "ipywidgets>=8.0.0",
    "requests", 
    "psutil",
    "pyngrok",
    "cloud-detect"
]

print("📦 Installing Python dependencies...")
for pkg in packages:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], check=False)

# Install system packages
print("🔧 Installing system dependencies...")
system_pkgs = ["wget", "curl", "git", "nodejs", "npm"]
for pkg in system_pkgs:
    subprocess.run(["apt-get", "install", "-y", "-q", pkg], 
                  capture_output=True, check=False)

print("✅ Dependencies installed")

📦 Installing Python dependencies...


🔧 Installing system dependencies...
✅ Dependencies installed


In [3]:
#@title 🎨 **Complete AI App Manager Interface**
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import threading
import time
import json
import requests

# Global state
pinokio_manager = None
app_processes = {}  # Track running AI apps
tunnel_urls = {}    # Track tunnel URLs for apps

class PinokioAppManager:
    def __init__(self):
        self.pinokio = None
        self.setup_complete = False
        self.server_running = False
        self.installed_apps = {}
        self.running_apps = {}
        self.create_interface()
    
    def create_interface(self):
        """Create the complete ipywidgets interface"""
        
        # Header
        self.header = widgets.HTML(f"""
        <div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); 
                    color: white; padding: 20px; border-radius: 10px; text-align: center; margin-bottom: 20px;">
            <h1>🚀 Pinokio AI App Manager</h1>
            <p>Complete AI Application Deployment & Management</p>
            <p><small>Using GitHub Repository Implementation</small></p>
        </div>
        """)
        
        # Status indicators
        self.status_indicator = widgets.HTML("<div style='color: red; font-weight: bold;'>● Offline</div>")
        self.info_text = widgets.HTML("<div>Ready to start...</div>")
        
        # Progress bar
        self.progress = widgets.IntProgress(
            value=0, min=0, max=100,
            description="Progress:",
            layout=widgets.Layout(width='100%'),
            style={'bar_color': '#1f77b4'}
        )
        
        # Main control buttons
        self.setup_btn = widgets.Button(
            description="🔧 Setup Pinokio",
            button_style='primary',
            layout=widgets.Layout(width='150px', height='35px')
        )
        
        self.start_btn = widgets.Button(
            description="🚀 Start Server", 
            button_style='success',
            layout=widgets.Layout(width='150px', height='35px'),
            disabled=True
        )
        
        self.stop_btn = widgets.Button(
            description="⏹️ Stop All",
            button_style='danger', 
            layout=widgets.Layout(width='150px', height='35px'),
            disabled=True
        )
        
        # Configuration
        self.port_input = widgets.IntText(
            value=42000,
            description="Port:",
            layout=widgets.Layout(width='200px')
        )
        
        # AI App Selection
        self.app_selector = widgets.Dropdown(
            options=[
                ('AUTOMATIC1111 WebUI', 'automatic1111'),
                ('ComfyUI', 'comfyui'),
                ('Text Generation WebUI', 'text-generation'),
                ('Fooocus', 'fooocus'),
                ('InvokeAI', 'invokeai'),
                ('Kohya SS', 'kohya-ss'),
                ('Open WebUI', 'open-webui'),
                ('Whisper', 'whisper'),
                ('FaceFusion', 'facefusion'),
                ('Real-ESRGAN', 'real-esrgan')
            ],
            description="AI App:",
            layout=widgets.Layout(width='300px')
        )
        
        self.install_app_btn = widgets.Button(
            description="📦 Install App",
            button_style='info',
            layout=widgets.Layout(width='120px', height='35px'),
            disabled=True
        )
        
        self.launch_app_btn = widgets.Button(
            description="🚀 Launch App", 
            button_style='success',
            layout=widgets.Layout(width='120px', height='35px'),
            disabled=True
        )
        
        self.stop_app_btn = widgets.Button(
            description="⏹️ Stop App",
            button_style='warning',
            layout=widgets.Layout(width='120px', height='35px'), 
            disabled=True
        )
        
        # Tunnel configuration
        self.tunnel_service = widgets.Dropdown(
            options=[
                ('Cloudflare (Free)', 'cloudflare'),
                ('ngrok (Token Required)', 'ngrok'),
                ('LocalTunnel (Free)', 'localtunnel')
            ],
            value='cloudflare',
            description="Tunnel:",
            layout=widgets.Layout(width='250px')
        )
        
        self.ngrok_token = widgets.Text(
            placeholder="ngrok token (if using ngrok)",
            description="Token:",
            layout=widgets.Layout(width='300px')
        )
        
        # App status display
        self.app_status = widgets.HTML(
            value="<div>No apps installed</div>"
        )
        
        # URLs display
        self.urls_display = widgets.HTML(
            value="<div>No active URLs</div>"
        )
        
        # Log output
        self.log_output = widgets.Output(
            layout=widgets.Layout(
                height='350px',
                border='1px solid #ccc',
                overflow_y='auto'
            )
        )
        
        # Bind events
        self.setup_btn.on_click(self.setup_pinokio)
        self.start_btn.on_click(self.start_server)
        self.stop_btn.on_click(self.stop_all)
        self.install_app_btn.on_click(self.install_app)
        self.launch_app_btn.on_click(self.launch_app)
        self.stop_app_btn.on_click(self.stop_app)
        
        # Layout
        self.create_layout()
    
    def create_layout(self):
        """Create the UI layout"""
        
        # Status section
        status_section = widgets.HBox([
            self.status_indicator,
            self.info_text
        ], layout=widgets.Layout(justify_content='space-between'))
        
        # Main controls
        main_controls = widgets.VBox([
            widgets.HTML("<h3>🎛️ Pinokio Controls</h3>"),
            self.progress,
            status_section,
            widgets.HBox([self.setup_btn, self.start_btn, self.stop_btn]),
            widgets.HBox([
                self.port_input,
                widgets.HTML("<div style='width: 20px;'></div>"),
                self.tunnel_service
            ]),
            self.ngrok_token
        ])
        
        # App management
        app_controls = widgets.VBox([
            widgets.HTML("<h3>🤖 AI App Management</h3>"),
            self.app_selector,
            widgets.HBox([self.install_app_btn, self.launch_app_btn, self.stop_app_btn]),
            widgets.HTML("<h4>📊 App Status</h4>"),
            self.app_status,
            widgets.HTML("<h4>🌐 Active URLs</h4>"), 
            self.urls_display
        ])
        
        # Main interface
        self.interface = widgets.VBox([
            self.header,
            widgets.HBox([
                main_controls,
                widgets.HTML("<div style='width: 30px;'></div>"),
                app_controls
            ], layout=widgets.Layout(align_items='flex-start')),
            widgets.HTML("<h3>📋 System Logs</h3>"),
            self.log_output
        ])
    
    def log(self, message, level="INFO"):
        """Add message to log output"""
        timestamp = time.strftime("%H:%M:%S")
        with self.log_output:
            if level == "ERROR":
                print(f"🔴 [{timestamp}] {message}")
            elif level == "SUCCESS":
                print(f"🟢 [{timestamp}] {message}")
            elif level == "WARNING":
                print(f"🟡 [{timestamp}] {message}")
            else:
                print(f"🔵 [{timestamp}] {message}")
    
    def update_status(self, status, info=""):
        """Update status indicators"""
        if status == "online":
            self.status_indicator.value = "<div style='color: green; font-weight: bold;'>● Online</div>"
        elif status == "installing":
            self.status_indicator.value = "<div style='color: orange; font-weight: bold;'>● Installing</div>"
        else:
            self.status_indicator.value = "<div style='color: red; font-weight: bold;'>● Offline</div>"
        
        self.info_text.value = f"<div><small>{info}</small></div>"
    
    def update_progress(self, value, description=""):
        """Update progress bar"""
        self.progress.value = value
        if description:
            self.progress.description = description
    
    def update_buttons(self, setup=None, start=None, stop=None, install=None, launch=None, stop_app=None):
        """Update button states"""
        if setup is not None:
            self.setup_btn.disabled = not setup
        if start is not None:
            self.start_btn.disabled = not start
        if stop is not None:
            self.stop_btn.disabled = not stop
        if install is not None:
            self.install_app_btn.disabled = not install
        if launch is not None:
            self.launch_app_btn.disabled = not launch
        if stop_app is not None:
            self.stop_app_btn.disabled = not stop_app
    
    def update_app_status(self):
        """Update app status display"""
        if not self.installed_apps and not self.running_apps:
            self.app_status.value = "<div>No apps installed</div>"
            return
        
        html = "<div>"
        
        # Installed apps
        if self.installed_apps:
            html += "<strong>📦 Installed:</strong><br>"
            for app_id, info in self.installed_apps.items():
                status = "🔴 Stopped"
                if app_id in self.running_apps:
                    status = "🟢 Running"
                html += f"• {info.get('name', app_id)}: {status}<br>"
        
        # Running apps
        if self.running_apps:
            html += "<br><strong>🚀 Running:</strong><br>"
            for app_id, info in self.running_apps.items():
                port = info.get('port', 'Unknown')
                html += f"• {info.get('name', app_id)}: Port {port}<br>"
        
        html += "</div>"
        self.app_status.value = html
    
    def update_urls(self):
        """Update URLs display"""
        if not tunnel_urls and not self.running_apps:
            self.urls_display.value = "<div>No active URLs</div>"
            return
        
        html = "<div>"
        
        # Pinokio server URL
        if self.server_running:
            port = self.port_input.value
            html += f"<strong>🎛️ Pinokio Server:</strong><br>"
            html += f"• Local: <a href='http://localhost:{port}' target='_blank'>http://localhost:{port}</a><br>"
        
        # App URLs
        if tunnel_urls:
            html += "<br><strong>🌐 Public App URLs:</strong><br>"
            for app_id, url in tunnel_urls.items():
                app_name = self.installed_apps.get(app_id, {}).get('name', app_id)
                html += f"• {app_name}: <a href='{url}' target='_blank'>{url}</a><br>"
        
        # Local app URLs
        if self.running_apps:
            html += "<br><strong>🏠 Local App URLs:</strong><br>"
            for app_id, info in self.running_apps.items():
                port = info.get('port')
                if port:
                    app_name = info.get('name', app_id)
                    local_url = f"http://localhost:{port}"
                    html += f"• {app_name}: <a href='{local_url}' target='_blank'>{local_url}</a><br>"
        
        html += "</div>"
        self.urls_display.value = html
    
    def setup_pinokio(self, btn):
        """Setup Pinokio environment"""
        self.log("🔧 Setting up Pinokio environment...")
        self.update_status("installing", "Setting up environment...")
        self.update_progress(0, "Setup:")
        self.update_buttons(setup=False, start=False, stop=False)
        
        def setup_thread():
            try:
                # Initialize Pinokio
                self.pinokio = PinokioCloudGPU(log_level="INFO")
                self.update_progress(25, "Setup:")
                
                # Setup environment
                if self.pinokio.setup():
                    self.update_progress(75, "Setup:")
                    
                    # Install Pinokio
                    if self.pinokio.install_pinokio():
                        self.update_progress(100, "Setup:")
                        self.setup_complete = True
                        self.log("✅ Pinokio setup completed successfully!", "SUCCESS")
                        self.update_status("offline", "Ready to start server")
                        self.update_buttons(setup=False, start=True, stop=True)
                    else:
                        raise Exception("Pinokio installation failed")
                else:
                    raise Exception("Environment setup failed")
                    
            except Exception as e:
                self.log(f"❌ Setup failed: {str(e)}", "ERROR")
                self.update_status("offline", "Setup failed")
                self.update_buttons(setup=True, start=False, stop=False)
        
        threading.Thread(target=setup_thread, daemon=True).start()
    
    def start_server(self, btn):
        """Start Pinokio server"""
        port = self.port_input.value
        self.log(f"🚀 Starting Pinokio server on port {port}...")
        self.update_status("installing", f"Starting server on port {port}")
        self.update_progress(0, "Starting:")
        self.update_buttons(start=False, stop=False)
        
        def start_thread():
            try:
                if self.pinokio.start_pinokio(port=port):
                    self.update_progress(100, "Starting:")
                    self.server_running = True
                    self.log("✅ Pinokio server started successfully!", "SUCCESS")
                    self.update_status("online", f"Server running on port {port}")
                    self.update_buttons(start=False, stop=True, install=True)
                    self.update_urls()
                else:
                    raise Exception("Server failed to start")
                    
            except Exception as e:
                self.log(f"❌ Server start failed: {str(e)}", "ERROR")
                self.update_status("offline", "Server start failed")
                self.update_buttons(start=True, stop=True)
        
        threading.Thread(target=start_thread, daemon=True).start()
    
    def install_app(self, btn):
        """Install selected AI app"""
        app_id = self.app_selector.value
        app_name = self.app_selector.label
        
        self.log(f"📦 Installing {app_name}...")
        self.update_progress(0, "Installing:")
        self.update_buttons(install=False, launch=False, stop_app=False)
        
        def install_thread():
            try:
                # Use the real Pinokio controller to install app
                if hasattr(self.pinokio, 'controller') and self.pinokio.controller:
                    # Install using controller
                    app_info = self.pinokio.controller.install_app(
                        f"https://github.com/{app_id}",
                        app_id
                    )
                    
                    if app_info:
                        self.installed_apps[app_id] = {
                            'name': app_name,
                            'info': app_info,
                            'installed_at': time.time()
                        }
                        
                        self.update_progress(100, "Installing:")
                        self.log(f"✅ {app_name} installed successfully!", "SUCCESS")
                        self.update_buttons(install=True, launch=True, stop_app=False)
                        self.update_app_status()
                    else:
                        raise Exception("App installation returned no info")
                else:
                    raise Exception("Pinokio controller not available")
                    
            except Exception as e:
                self.log(f"❌ {app_name} installation failed: {str(e)}", "ERROR")
                self.update_buttons(install=True, launch=False, stop_app=False)
        
        threading.Thread(target=install_thread, daemon=True).start()
    
    def launch_app(self, btn):
        """Launch selected AI app with tunnel"""
        app_id = self.app_selector.value
        app_name = self.app_selector.label
        tunnel_service = self.tunnel_service.value
        ngrok_token = self.ngrok_token.value or None
        
        if app_id not in self.installed_apps:
            self.log(f"❌ {app_name} is not installed", "ERROR")
            return
        
        self.log(f"🚀 Launching {app_name} with {tunnel_service} tunnel...")
        self.update_progress(0, "Launching:")
        self.update_buttons(launch=False, stop_app=False)
        
        def launch_thread():
            try:
                # Launch app using controller
                if hasattr(self.pinokio, 'controller') and self.pinokio.controller:
                    launched_app = self.pinokio.controller.launch_app(app_id, wait_for_ready=True)
                    
                    if launched_app and launched_app.url:
                        # App launched successfully
                        app_port = launched_app.port
                        
                        self.running_apps[app_id] = {
                            'name': app_name,
                            'port': app_port,
                            'local_url': launched_app.url,
                            'launched_at': time.time()
                        }
                        
                        self.update_progress(50, "Launching:")
                        self.log(f"✅ {app_name} launched on port {app_port}", "SUCCESS")
                        
                        # Setup tunnel for the app
                        self.log(f"🌐 Setting up {tunnel_service} tunnel for {app_name}...")
                        
                        tunnel_manager = TunnelManager()
                        tunnel_url = tunnel_manager.start_tunnel(
                            port=app_port,
                            service=tunnel_service,
                            ngrok_token=ngrok_token if tunnel_service == 'ngrok' else None
                        )
                        
                        if tunnel_url:
                            tunnel_urls[app_id] = tunnel_url
                            self.log(f"✅ {app_name} accessible at: {tunnel_url}", "SUCCESS")
                        else:
                            self.log(f"⚠️ Tunnel failed for {app_name}, using local access", "WARNING")
                        
                        self.update_progress(100, "Launching:")
                        self.update_buttons(launch=True, stop_app=True)
                        self.update_app_status()
                        self.update_urls()
                    else:
                        raise Exception("App launch failed")
                else:
                    raise Exception("Pinokio controller not available")
                    
            except Exception as e:
                self.log(f"❌ {app_name} launch failed: {str(e)}", "ERROR")
                self.update_buttons(launch=True, stop_app=False)
        
        threading.Thread(target=launch_thread, daemon=True).start()
    
    def stop_app(self, btn):
        """Stop selected AI app"""
        app_id = self.app_selector.value
        app_name = self.app_selector.label
        
        if app_id not in self.running_apps:
            self.log(f"❌ {app_name} is not running", "ERROR")
            return
        
        self.log(f"⏹️ Stopping {app_name}...")
        self.update_buttons(stop_app=False)
        
        def stop_thread():
            try:
                # Stop app using controller
                if hasattr(self.pinokio, 'controller') and self.pinokio.controller:
                    success = self.pinokio.controller.stop_app(app_id)
                    
                    if success:
                        # Remove from running apps
                        if app_id in self.running_apps:
                            del self.running_apps[app_id]
                        
                        # Remove tunnel URL
                        if app_id in tunnel_urls:
                            del tunnel_urls[app_id]
                        
                        self.log(f"✅ {app_name} stopped successfully", "SUCCESS")
                        self.update_buttons(stop_app=False, launch=True)
                        self.update_app_status()
                        self.update_urls()
                    else:
                        raise Exception("App stop command failed")
                else:
                    raise Exception("Pinokio controller not available")
                    
            except Exception as e:
                self.log(f"❌ Failed to stop {app_name}: {str(e)}", "ERROR")
                self.update_buttons(stop_app=True)
        
        threading.Thread(target=stop_thread, daemon=True).start()
    
    def stop_all(self, btn):
        """Stop all services"""
        self.log("⏹️ Stopping all services...")
        self.update_status("offline", "Shutting down...")
        self.update_progress(0, "Stopping:")
        self.update_buttons(setup=False, start=False, stop=False, install=False, launch=False, stop_app=False)
        
        def stop_thread():
            try:
                if self.pinokio:
                    self.pinokio.cleanup()
                
                # Clear state
                self.running_apps.clear()
                tunnel_urls.clear()
                self.server_running = False
                
                self.update_progress(100, "Stopping:")
                self.log("✅ All services stopped", "SUCCESS")
                self.update_status("offline", "All services stopped")
                self.update_buttons(setup=True, start=False, stop=False, install=False, launch=False, stop_app=False)
                self.update_app_status()
                self.update_urls()
                
            except Exception as e:
                self.log(f"⚠️ Cleanup error: {str(e)}", "WARNING")
                self.update_buttons(setup=True, start=False, stop=False)
        
        threading.Thread(target=stop_thread, daemon=True).start()
    
    def display(self):
        """Display the interface"""
        display(self.interface)

# Create and display the interface
print("🎨 Creating AI App Manager Interface...")
app_manager = PinokioAppManager()
app_manager.display()
print("✅ Interface ready! Use the controls above to manage AI apps.")

🎨 Creating AI App Manager Interface...


✅ Interface ready! Use the controls above to manage AI apps.


In [4]:
#@title 🚀 **Quick Setup Functions (Optional)**

def quick_setup_and_start():
    """Quick setup function for experienced users"""
    global app_manager
    
    print("🚀 QUICK SETUP & START")
    print("=" * 30)
    
    def auto_setup():
        # Simulate clicking setup and start buttons
        if app_manager:
            app_manager.setup_pinokio(None)
            # Wait for setup to complete
            import time
            while not app_manager.setup_complete:
                time.sleep(1)
            
            # Start server
            app_manager.start_server(None)
            
            print("✅ Quick setup completed!")
            print("🎛️ Use the interface above to install and launch AI apps")
    
    import threading
    threading.Thread(target=auto_setup, daemon=True).start()

def install_popular_apps():
    """Install popular AI apps"""
    popular_apps = ['automatic1111', 'comfyui', 'fooocus']
    
    if not app_manager or not app_manager.server_running:
        print("❌ Please setup and start Pinokio server first")
        return
    
    print("📦 Installing popular AI apps...")
    for app_id in popular_apps:
        try:
            # Set dropdown to app and install
            for option in app_manager.app_selector.options:
                if option[1] == app_id:
                    app_manager.app_selector.value = app_id
                    app_manager.install_app(None)
                    break
            
            # Wait for installation
            import time
            time.sleep(30)  # Adjust based on your needs
            
        except Exception as e:
            print(f"❌ Failed to install {app_id}: {e}")

# Uncomment to use quick functions:
# quick_setup_and_start()
# install_popular_apps()

print("⚡ Quick setup functions available:")
print("   - quick_setup_and_start(): Auto setup and start")
print("   - install_popular_apps(): Install popular apps")

⚡ Quick setup functions available:
   - quick_setup_and_start(): Auto setup and start
   - install_popular_apps(): Install popular apps


In [5]:
#@title 📊 **Advanced Monitoring & Status**
import ipywidgets as widgets
from IPython.display import display
import time
import threading

def create_monitoring_interface():
    """Create advanced monitoring interface"""
    
    # Create widgets
    refresh_btn = widgets.Button(
        description="🔄 Refresh",
        button_style='info',
        layout=widgets.Layout(width='100px')
    )
    
    auto_refresh = widgets.Checkbox(
        value=False,
        description="Auto-refresh (5s)"
    )
    
    status_output = widgets.Output(
        layout=widgets.Layout(
            height='300px',
            border='1px solid #ccc',
            overflow_y='auto'
        )
    )
    
    def update_monitor():
        """Update monitoring display"""
        with status_output:
            status_output.clear_output(wait=True)
            
            print("📊 SYSTEM MONITORING")
            print("=" * 40)
            
            # Pinokio status
            if app_manager and app_manager.pinokio:
                try:
                    status = app_manager.pinokio.get_status()
                    
                    print("🎛️ PINOKIO STATUS:")
                    print(f"   State: {status['deployment']['state']}")
                    print(f"   Platform: {status['deployment']['platform']}")
                    
                    pinokio_status = status['pinokio']
                    running = "✅ Running" if pinokio_status['running'] else "❌ Stopped"
                    print(f"   Server: {running}")
                    if pinokio_status['port']:
                        print(f"   Port: {pinokio_status['port']}")
                    
                    # Tools status
                    tools = status['tools']
                    print(f"\n🤖 INSTALLED APPS: {tools['count']}")
                    for tool in tools['installed']:
                        print(f"   • {tool}")
                    
                except Exception as e:
                    print(f"❌ Status error: {e}")
            else:
                print("🎛️ PINOKIO STATUS: Not initialized")
            
            # App status
            if app_manager:
                print(f"\n📦 INSTALLED APPS: {len(app_manager.installed_apps)}")
                for app_id, info in app_manager.installed_apps.items():
                    status = "🟢 Running" if app_id in app_manager.running_apps else "🔴 Stopped"
                    print(f"   • {info['name']}: {status}")
                
                print(f"\n🚀 RUNNING APPS: {len(app_manager.running_apps)}")
                for app_id, info in app_manager.running_apps.items():
                    port = info.get('port', 'Unknown')
                    tunnel = "🌐 Tunneled" if app_id in tunnel_urls else "🏠 Local"
                    print(f"   • {info['name']}: Port {port} ({tunnel})")
                
                print(f"\n🌐 TUNNEL URLS: {len(tunnel_urls)}")
                for app_id, url in tunnel_urls.items():
                    app_name = app_manager.installed_apps.get(app_id, {}).get('name', app_id)
                    print(f"   • {app_name}: {url}")
            
            print(f"\n🕐 Last updated: {time.strftime('%H:%M:%S')}")
    
    def on_refresh_click(btn):
        update_monitor()
    
    def auto_refresh_loop():
        while auto_refresh.value:
            update_monitor()
            time.sleep(5)
    
    def on_auto_change(change):
        if change['new']:
            threading.Thread(target=auto_refresh_loop, daemon=True).start()
    
    # Bind events
    refresh_btn.on_click(on_refresh_click)
    auto_refresh.observe(on_auto_change, names='value')
    
    # Initial update
    update_monitor()
    
    # Layout
    monitor_ui = widgets.VBox([
        widgets.HTML("<h3>📊 System Monitor</h3>"),
        widgets.HBox([refresh_btn, auto_refresh]),
        status_output
    ])
    
    return monitor_ui

# Create and display monitoring interface
monitor = create_monitoring_interface()
display(monitor)

In [6]:
#@title 🛠️ **App Management Utilities**
import ipywidgets as widgets
from IPython.display import display
import json
from datetime import datetime

def create_app_utilities():
    """Create app management utilities"""
    
    # Backup/restore functions
    backup_btn = widgets.Button(
        description="💾 Backup Config",
        button_style='success',
        layout=widgets.Layout(width='150px')
    )
    
    restore_btn = widgets.Button(
        description="📁 Load Config",
        button_style='info', 
        layout=widgets.Layout(width='150px')
    )
    
    # Bulk operations
    install_all_btn = widgets.Button(
        description="📦 Install All Popular",
        button_style='warning',
        layout=widgets.Layout(width='180px')
    )
    
    stop_all_apps_btn = widgets.Button(
        description="⏹️ Stop All Apps",
        button_style='danger',
        layout=widgets.Layout(width='150px')
    )
    
    # Utilities output
    utils_output = widgets.Output(
        layout=widgets.Layout(height='200px', border='1px solid #ccc')
    )
    
    def backup_config():
        """Backup current configuration"""
        with utils_output:
            if app_manager and app_manager.pinokio:
                try:
                    config_data = {
                        'timestamp': datetime.now().isoformat(),
                        'installed_apps': app_manager.installed_apps,
                        'running_apps': app_manager.running_apps,
                        'tunnel_urls': tunnel_urls,
                        'server_running': app_manager.server_running,
                        'port': app_manager.port_input.value,
                        'tunnel_service': app_manager.tunnel_service.value
                    }
                    
                    filename = f"pinokio_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
                    
                    with open(filename, 'w') as f:
                        json.dump(config_data, f, indent=2, default=str)
                    
                    print(f"✅ Configuration backed up to: {filename}")
                    
                except Exception as e:
                    print(f"❌ Backup failed: {e}")
            else:
                print("❌ No configuration to backup")
    
    def install_all_popular():
        """Install all popular apps"""
        with utils_output:
            if not app_manager or not app_manager.server_running:
                print("❌ Please setup and start Pinokio server first")
                return
            
            popular = ['automatic1111', 'comfyui', 'fooocus', 'text-generation']
            print(f"📦 Installing {len(popular)} popular apps...")
            
            for app_id in popular:
                try:
                    # Find and select the app
                    for option in app_manager.app_selector.options:
                        if option[1] == app_id:
                            app_manager.app_selector.value = app_id
                            print(f"🔄 Installing {option[0]}...")
                            app_manager.install_app(None)
                            break
                    
                    time.sleep(10)  # Wait between installs
                    
                except Exception as e:
                    print(f"❌ Failed to install {app_id}: {e}")
            
            print("✅ Bulk installation initiated")
    
    def stop_all_apps():
        """Stop all running apps"""
        with utils_output:
            if not app_manager:
                print("❌ No app manager available")
                return
            
            running_apps = list(app_manager.running_apps.keys())
            if not running_apps:
                print("ℹ️ No apps are currently running")
                return
            
            print(f"⏹️ Stopping {len(running_apps)} apps...")
            
            for app_id in running_apps:
                try:
                    app_name = app_manager.running_apps[app_id]['name']
                    app_manager.app_selector.value = app_id
                    print(f"🔄 Stopping {app_name}...")
                    app_manager.stop_app(None)
                    time.sleep(2)
                    
                except Exception as e:
                    print(f"❌ Failed to stop {app_id}: {e}")
            
            print("✅ All apps stopped")
    
    # Bind events
    backup_btn.on_click(lambda x: backup_config())
    install_all_btn.on_click(lambda x: install_all_popular())
    stop_all_apps_btn.on_click(lambda x: stop_all_apps())
    
    # Layout
    utils_ui = widgets.VBox([
        widgets.HTML("<h3>🛠️ App Management Utilities</h3>"),
        widgets.HBox([
            backup_btn, restore_btn, 
            widgets.HTML("<div style='width: 20px;'></div>"),
            install_all_btn, stop_all_apps_btn
        ]),
        utils_output
    ])
    
    return utils_ui

# Create and display utilities
utilities = create_app_utilities()
display(utilities)

print("\n🎉 COMPLETE PINOKIO IPYWIDGETS INTERFACE READY!")
print("=" * 60)
print("✅ All components loaded")
print("🎛️ Main interface: Setup → Start → Install Apps → Launch Apps")
print("📊 Real-time monitoring included")
print("🛠️ Bulk management utilities available")
print("🌐 Automatic tunneling for all apps")
print("\n🚀 Ready to deploy AI applications!")


🎉 COMPLETE PINOKIO IPYWIDGETS INTERFACE READY!
✅ All components loaded
🎛️ Main interface: Setup → Start → Install Apps → Launch Apps
📊 Real-time monitoring included
🛠️ Bulk management utilities available
🌐 Automatic tunneling for all apps

🚀 Ready to deploy AI applications!
